# Краулеры

**План**

1. Что такое краулеры?
2. Как написать простой краулер?
3. Блокировки и способы их обхода

## Что такое краулеры?

Краулеры - это боты/программы, которые "ползают" по страницам сайта и собирают информацию. Все чаще использование таких программ запрещается правилами пользования сайтами, поэтому это формально нехорошо. Но так продолжают делать и это надо уметь. Запрещают по 2 основным причинам: не хотят делиться данными и боятся, что вы уроните сервер (если сайт маленький и сервер не очень, то это довольно легко). Поэтому нужно собирать данные аккуратно, чтобы вас а) не заблокировали по IP и б) вы не навредили серверу

## Как написать простой краулер?

In [1]:
import requests
from pprint import pprint

Вместо отдельных запросов лучше создать сессию, которая позволит хранить информацию между запросами и поддерживать то же соединение и не создавать каждый раз все заново, что влияет на производительность.

In [2]:
session = requests.session()

Попробуем сделать запрос, просто вместо requests.get мы пишет session.get.

In [3]:
response = session.get('https://ru.wikipedia.org')

Может найти значения нашего IP-адреса

In [4]:
response.headers['X-Client-IP']

'109.252.208.253'

Можно ли нас отследить по IP? С определенной точностью, можно узнать округ или компанию, к которой привязан любой IP. Пример сервиса, который позволяет это сделать [здесь](https://whatismyipaddress.com/ip-lookup)

Посмотреть на headers запроса

In [5]:
pprint(dict(response.headers))

{'accept-ch': '',
 'accept-ranges': 'bytes',
 'age': '511',
 'cache-control': 'private, s-maxage=0, max-age=0, must-revalidate',
 'content-encoding': 'gzip',
 'content-language': 'ru',
 'content-length': '28213',
 'content-type': 'text/html; charset=UTF-8',
 'date': 'Wed, 08 Nov 2023 20:01:26 GMT',
 'last-modified': 'Wed, 08 Nov 2023 20:01:23 GMT',
 'nel': '{ "report_to": "wm_nel", "max_age": 604800, "failure_fraction": 0.05, '
        '"success_fraction": 0.0}',
 'report-to': '{ "group": "wm_nel", "max_age": 604800, "endpoints": [{ "url": '
              '"https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" '
              '}] }',
 'server': 'mw1371.eqiad.wmnet',
 'server-timing': 'cache;desc="hit-front", host;desc="cp3072"',
 'set-cookie': 'WMF-DP=68f;Path=/;HttpOnly;secure;Expires=Thu, 09 Nov 2023 '
               '00:00:00 GMT, '
               'NetworkProbeLimit=0.001;Path=/;Secure;Max-Age=3600',
 '

### Стратегии сбора данных


По сути краулеры выполняют сбор страниц (их html) как мы это делали на прошлом занятии, но делают они это циклами (или циклами циклов). Можно выделить разные стратегии сбора данных:
    
**По типу навигации**

1. Все страницы со ссылками имеют удобные номера ("https://ficbook.net/fanfiction/no_fandom/originals?p=2"), обычно просто p=(число) или page=(число). В этом случае вам нужно просто подставлять цифры подробнее про параметры передаваемые в ссылке можно посмотреть [здесь](https://en.wikipedia.org/wiki/Query_string)
2. Страницы называются как-то не структурированно (например, по названиям блоков). Тут нужно собирать ссылки на эти страницы и потом по ним ходить и собирать конечные странички.
3. Все расположено на одной страничке и догружается с использованием [WebSocket](https://en.wikipedia.org/wiki/WebSocket) или других технологий, при адрес в адресной строке никак не изменяется, данные могут догружаться на сайт автоматически по мере скролла страницы

**По скорости обновления**

1. Если сайт довольно статичный по контенту (медленно появляются и удаляются материалы), то можно сначал собрать ссылки, а потом по ним ходить
2. Если сайт очень динамичный по контенту (например, объявления на крупном сайте), вам нужно при получении страничкии ссылок сразу их обходить, а потом переходить к следующей, потому что ко времени получения исчерпывающего списка ссылок по сайту многие будут уже удалены или недоступны



## Блокировки и способы их обхода

Для того, чтобы предотвратить автоматический сбор информации с некого сайта, применяются различные инструменты, которые определяют роботов и блокируют запросы с адресов, которые были классифицированы как роботы. Чтобы не заблокировали домашний/учебный ip, лучше сразу задуматься об этих мерах и предотвратить возможные проблемы. Кстати, Википедия не блокирует и можно спокойно скачивать без каких-либо проблем.

Чтобы их обойти, можно попробовать несколько инструментов:
1. time.sleep(x) - задержка между запросами, чтобы слишком большая скорость запросов не показалась подозрительной или ваши запросы не уронили сервер небольшого ресурса (например, региональной газеты)
2. time.sleep(случайный промежуток времени) - это более хитрая версия, когда время задержки - это случайное число из некоторого отрезка (модуль random)
3. изобразить браузер - при запросе отправляется информация о том, из какого приложения пришел запрос (например, Googlr Chrome), запросы сделанные из браузера больше похожи на человеческие, для этого нужно задать user-agent в параметрах (а его выбирать случайно с помощью fake_useragent)
4. использовать прокси - существуют ресурсы с бесплатными списками открытых прокси, через которые можно пропускать ваш запрос и сервер будет думать, что запросы приходят из разных мест (anonymous и elite классы прокси) или использовать анонимизированные сети к примеру сеть [Tor](https://en.wikipedia.org/wiki/Tor_(network)) и аналоги.

### Пауза между запросами

In [6]:
import time
from datetime import datetime

In [7]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(datetime.now())
    time.sleep(1)

2023-11-08 23:10:00.261682
2023-11-08 23:10:01.394032
2023-11-08 23:10:02.523687
2023-11-08 23:10:03.646711
2023-11-08 23:10:04.786949


### Притвориться нормальным браузером

In [8]:
from fake_useragent import UserAgent

Можно настроить так, чтобы не проверять безопасность соединения, что иногда вызывает ошибки. Но это можно делать с сайтмаи, которым вы доверяете.

In [9]:
ua = UserAgent()

In [10]:
headers = {'User-Agent': ua.random}
print(headers)
response = session.get('https://ru.wikipedia.org', headers=headers)

{'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0'}


### Пауза между запросами (случайное время)

In [11]:
import random

random.uniform позволяет получить случайное число из отрезка

In [12]:
random.uniform(1, 3)

2.1792623898189505

In [13]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(datetime.now())
    time.sleep(random.uniform(1.1, 5.2))

2023-11-08 23:10:06.287758
2023-11-08 23:10:08.662543
2023-11-08 23:10:12.928754
2023-11-08 23:10:16.976760
2023-11-08 23:10:18.895382


### Подключение через прокси

Прокси-сервер — это дополнительное звено между вами и интернетом, через него пойдет подключение и сайт не будет знать, что это вы посылаете запрос.

Адреса прокси можно взять со специальных сайтов, например, [https://hideip.me/ru/proxy/httplist](https://hideip.me/ru/proxy/httplist). И потом проверить, что они рабочие, прежде чем использовать [https://checkerproxy.net/](https://checkerproxy.net/)

In [14]:
proxy = {"https://": "https://176.213.141.107:8080"}

response = session.get('http://ru.wikipedia.org', proxies=proxy)
print(response.headers['X-Client-IP'])

109.252.208.253


## Примеры

### Пример 1

Давайте обкачаем немного новостей с сайта вышки.

1. Страницы имеют вид "https://www.hse.ru/news/page1.html", поэтому можно просто идти циклом.
2. Достанем дату публикации, заголовок, краткое описание (из станицы со списком новостей), текст полной статьи и метки (из самой страницы новости)
3. Сохраним в датафрейм

In [15]:
from bs4 import BeautifulSoup
import re

In [16]:
texts, tags, pub_year, pub_month, pub_dat, title,short_text, full_text,

NameError: name 'texts' is not defined

### Отработаем процесс на одной странице

**Шаг 1. Найти страницы**

Посмотрим, как устроены новости и скачаем одну страницу

In [ ]:
page_number = 1
url = f'https://www.hse.ru/news/page{page_number}.html'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

Распарсим с помощью BeautifulSoup

In [ ]:
soup = BeautifulSoup(page, 'html.parser')

Найдем отдельные посты

In [ ]:
news = soup.find_all('div', {'class': 'post'})

In [ ]:
len(news)

Найдем заголовок-ссылку и запомним текст заголовка

In [ ]:
title_obj = news[0].find('a')
title_obj

In [ ]:
title = title_obj.text
title

Достанем свойства этой ссылки (куда ведет, class)

In [ ]:
attrs = title_obj.attrs
attrs

Достанем саму ссылку

In [ ]:
href = title_obj.attrs['href']
href

Достанем текст новости

In [ ]:
short_text = news[0].find('div', {'class': 'post__text'}).text
short_text

Достанем день, месяц, год публикации

In [ ]:
pub_day = news[0].find('div', {'class': 'post-meta__day'}).text
pub_day

In [ ]:
pub_month = news[0].find('div', {'class': 'post-meta__month'}).text
pub_month

In [ ]:
pub_year = news[0].find('div', {'class': 'post-meta__year'}).text
pub_year

**Шаг 2. Научиться парсить страничку самой новости**

Возьмем ссылку на полную новость и соединим с адерсом сайта (т.к. ссылка относительная)

In [ ]:
url_one = 'http://www.hse.ru' + href
url_one

Скачаем ее и распарсим

In [ ]:
req = session.get(url_one, headers={'User-Agent': ua.random})
page = req.text

soup = BeautifulSoup(page, 'html.parser')

Сохраним текст, распечатаем кусочек

In [ ]:
full_text = soup.find('div', {'class': 'post__content'}).text
full_text[:200]

Найдем теги, которые присвоены статье

In [ ]:
meta = soup.find('div', {'class': 'articleMeta'})

tags = meta.find_all('a', {'class': 'tag'})
tags = [t.text for t in tags]
tags

**Шаг 3. Оформляем нормально в функции**

Сделаем словарь соответствий имени месяца и его номера

In [ ]:
months = {
    value: key + 1
    for key, value in enumerate(
        ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    )
}

Парсим информацию из страницы со списком новостей (блок одной новости)

In [ ]:
def parse_news_page_block(one_block):
    block = {}
    a = one_block.find('a')
    block['title'] = a.text
    block['href'] = a.attrs['href']
    block['short_text'] = one_block.find('div', {'class': 'post__text'}).text
    block['pub_day'] = int(one_block.find('div', {'class': 'post-meta__day'}).text)
    block['pub_month'] = months[one_block.find('div', {'class': 'post-meta__month'}).text]
    block['pub_year'] = int(one_block.find('div', {'class': 'post-meta__year'}).text)
    return block

Парсим отдельную страницу новости

In [ ]:
def parse_one_article(block):
    url_one = 'http://www.hse.ru' + block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block['full_text'] = soup.find('div', {'class': 'post__content'}).text
    meta = soup.find('div', {'class': 'articleMeta'})
    tags = meta.find_all('a', {'class': 'tag'})
    block['tags'] = [t.text for t in tags]     
    return block

Регулярное выражение для того, чтобы достать ID новости и не повторяться

In [ ]:
regex_hse_id = re.compile('/([0-9]*?).html')

Обработать N-ую страницу новостей

In [ ]:
def get_nth_page(page_number, seen_news):
    # скачиваем
    url = f'https://www.hse.ru/news/page{page_number}.html'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    
    # находим новости
    news = soup.find_all('div', {'class': 'post'})
    
    # идем по новостям и обрабатываем их
    blocks = []
    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
            print(e)

    # идем по отдельным статьям и достаем информацию
    result = []
    for b in blocks:
        if b['href'].startswith('/'):
            idx = regex_hse_id.findall(b['href'])[0]
            try:
                res = parse_one_article(b)
                res['hse_id'] = idx
                result.append(res)
            except Exception as e:
                print(e)
    
    # возвращаем найденную информацию
    return result

**Шаг 4. Сохраняем в датафрейм**


In [ ]:
from tqdm.auto import tqdm

Напишем функцию, куда передаем количество страниц и она выполняет все нужные действия

In [ ]:
import pandas as pd

In [ ]:
def run_all(n_pages):
    blocks = []
    for i in tqdm(range(n_pages)):
        blocks.extend(get_nth_page(i+1))
    
    return blocks

Запускаем на 20 первых страниц

In [ ]:
blocks = run_all(20)

df = pd.DataFrame(blocks)

Посмотрим на 10 самых популярных тегов

In [ ]:
from collections import Counter

In [ ]:
tags = [tag for tag_list in df["tags"] for tag in tag_list]

In [ ]:
for title, counts in sorted(Counter(tags).items(), key=lambda x: -x[-1])[:10]:
    print(counts,"\t", title)

Посмотрим, сколько публикаций по месяцам

In [ ]:
df.groupby("pub_month")["hse_id"].count()